In [71]:
import robin_stocks
import robin_stocks.robinhood as r
import pandas as pd
from configs import *
login = r.login(username, pwd)

In [72]:
final_table_columns = ['chain_symbol','expiration_date', 'rhs_tradability', 
                        'state', 'strike_price', 'tradability', 'type', 
                        'ask_price', 'bid_price', 'break_even_price',
                        'instrument', 'open_interest', 'volume' ,'symbol', 'chance_of_profit_long',
                        'chance_of_profit_short', 'delta', 'gamma', 'implied_volatility',
                        'rho', 'theta', 'vega']

In [73]:
optionData = r.find_options_by_expiration(
        'tsla', 
        expirationDate='2021-05-21',
        optionType='call'
    )

df = pd.DataFrame(optionData)
df = df.drop(columns=[col for col in df if col not in final_table_columns])

Found Additional pages.
Loading page 2 ...
Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Da

In [74]:
df.head()

,chain_symbol,expiration_date,rhs_tradability,state,strike_price,tradability,type,ask_price,bid_price,break_even_price,...,volume,symbol,chance_of_profit_long,chance_of_profit_short,delta,gamma,implied_volatility,rho,theta,vega
0,TSLA,2021-05-21,untradable,active,905.0000,tradable,call,0.130000,0.060000,905.100000,...,37,TSLA,0.002160,0.997840,0.003602,0.000114,1.133660,0.000419,-0.065475,0.009018
1,TSLA,2021-05-21,untradable,active,925.0000,tradable,call,0.110000,0.050000,925.080000,...,54,TSLA,0.001692,0.998308,0.002881,0.000091,1.156152,0.000335,-0.054622,0.007377
2,TSLA,2021-05-21,untradable,active,990.0000,tradable,call,0.080000,0.030000,990.060000,...,70,TSLA,0.001115,0.998885,0.002037,0.000061,1.266190,0.000236,-0.043712,0.005390
3,TSLA,2021-05-21,untradable,active,555.0000,tradable,call,34.150000,33.500000,588.830000,...,270,TSLA,0.372824,0.627176,0.627703,0.006048,0.748167,0.069525,-1.516942,0.316329
4,TSLA,2021-05-21,untradable,active,1525.0000,tradable,call,0.010000,0.000000,1525.010000,...,0,TSLA,0.000094,0.999906,0.000269,0.000006,1.864959,0.000031,-0.009996,0.000837
